In [23]:
#################################################
#################################################
## create schema in flat files first then load to sql
#################################################
#################################################

In [24]:
import pandas as pd
import numpy as np
import os
import logging
#
pd.set_option('max_columns', 500)
pd.set_option('max_colwidth', -1)
pd.set_option('max_rows', 500)
#
## https://www.patricksoftwareblog.com/python-logging-tutorial/
logging.basicConfig(level=logging.WARNING, filename='LOG_schemaCreationVer3.log',         \
    filemode='w', format='')
#    filemode='w', format='%(asctime)s %(levelname)s:%(message)s')
#
logging.warning(f"\n    *************************************************************** \n    *************************************************************** \n    *************************************************************** ")
logging.warning(f"\n    *************************************************************** \n    *************************************************************** \n    *************************************************************** ")
logging.warning(f"\n    *************************************************************** \n    *************************************************************** \n    *************************************************************** ")
logging.warning(f" Data Curation and Modelling Processing.")
logging.warning(f"\n    *************************************************************** \n    *************************************************************** \n    *************************************************************** ")
logging.warning(f"\n    *************************************************************** \n    *************************************************************** \n    *************************************************************** ")
logging.warning(f"\n    *************************************************************** \n    *************************************************************** \n    ***************************************************************  \n\n")

In [25]:
inFilePath = '/home/rohit/PyWDUbuntu/DM2DataCuration/'
inDataFile = 'Athlete_Events_CLEANDED_MissingReplaced_20191113_2258.csv'
#
## -1 means read all rows
rowsToRead = -1
#rowsToRead = 5
## load all columns into dataframe as string (object) by specifying the
##      dtype=str during loading, all the columns dtpye is object
#
if rowsToRead == -1:
    dfin = pd.read_csv(inFilePath+inDataFile, sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
else:
    dfin = pd.read_csv(inFilePath+inDataFile, sep=',', dtype=str, header=0, \
                       nrows=rowsToRead, keep_default_na=False, low_memory=False )
#
logging.warning(f"\n\n    *************************** \n    *************************** ")
logging.warning(f"\n    *************************** \n    *************************** \n\n")
print(f'# Rows read into dataframe = {rowsToRead if rowsToRead != -1 else "all the rows"}')
logging.warning(f'# Rows read into dataframe = {rowsToRead if rowsToRead != -1 else "all the rows"}')
#
print(f'NOTE: All columns are read in with dtype as str, i.e. all columns are of object type')
logging.warning(f'NOTE: All columns are read in with dtype as str, i.e. all columns are of object type')
print(f'NOTE: keep_default_na=False is used, so all data is loaded as it is without automatic replacement with null or Nan')
logging.warning(f'NOTE: keep_default_na=False is used, so all data is loaded as it is without automatic replacement with null or Nan')
#

# Rows read into dataframe = all the rows
NOTE: All columns are read in with dtype as str, i.e. all columns are of object type
NOTE: keep_default_na=False is used, so all data is loaded as it is without automatic replacement with null or Nan


In [26]:
dfin.shape

(255692, 16)

In [27]:
#################################################
#################################################
## create schema starts
#################################################
#################################################

In [28]:
outSchemaPath = '/home/rohit/PyWDUbuntu/DM2DataCuration/Schema/'

In [29]:
## ################
## Create Dimension Table for trivial cases -- having only Unique Key and Value       ################
##
## Applies to these columns: Season, City, Year, Team, Sport, Medals, NOC
##
dimTableTrivialCaseColumnList = ['Season', 'City', 'Year', 'Team', 'Sport', 'Event', 'Medal', 'NOC']
#
for colTrivial in dimTableTrivialCaseColumnList:
    dfTemp = dfin[[colTrivial]]
    #
    logging.warning(f"\n\n    *************************** \n    *************************** ")
    logging.warning(f"\n    *************************** \n    *************************** \n\n")
    #
    print(f"\nACTION:\nCreating Dimension Table for column {colTrivial}")
    logging.warning(f"\nACTION:\nCreating Dimension Table for column {colTrivial}")
    #
    print(f"Shape before: {dfTemp.shape}")
    logging.warning(f"Shape before: {dfTemp.shape}")
    #
    dfTemp.sort_values(by=[colTrivial], ascending=True, inplace=True, na_position='first')
    dfTemp.drop_duplicates(subset= [colTrivial], keep='last', inplace=True)
    dfTemp = dfTemp.reset_index(drop=True)
    dfTemp = dfTemp.reset_index()
    newColName = 'PK_' + colTrivial
    dfTemp.rename(columns={'index' : newColName}, inplace=True)
    dfTemp[newColName] = dfTemp[newColName] + 1
    #
    print(f"Shape after: {dfTemp.shape}")
    logging.warning(f"Shape after: {dfTemp.shape}")
    #
    outFile = 'Dim_' + colTrivial + '.csv'
    dfTemp.to_csv(outSchemaPath + outFile, index=False)
    #
    print(f"Dimension Table created here:\n{outSchemaPath + outFile}")
    logging.warning(f"Dimension Table created here:\n{outSchemaPath + outFile}")
#
del dfTemp, newColName, dimTableTrivialCaseColumnList
#
#
#
## ################
## Create Dimension Table for Games       ################
##
#
## read in dimension tables for: Season, City, Year
seasonDimFile = 'Dim_Season.csv'
cityDimFile = 'Dim_City.csv'
yearDimFile = 'Dim_Year.csv'
#
## load all columns into dataframe as string (object) by specifying the
##      dtype=str during loading, all the columns dtpye is object
#
dfDimSeason = pd.read_csv(outSchemaPath+seasonDimFile, sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
dfDimCity   = pd.read_csv(outSchemaPath+cityDimFile,   sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
dfDimYear   = pd.read_csv(outSchemaPath+yearDimFile,   sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
#
logging.warning(f"\n\n    *************************** \n    *************************** ")
logging.warning(f"\n    *************************** \n    *************************** \n\n")
#
print(f"\nACTION:\nCreating Dimension Table for column Game.")
logging.warning(f"\nACTION:\nCreating Dimension Table for column Game.")
#
dfTemp = dfin[['Year', 'Season', 'City']]
#
print(f"Shape before: {dfTemp.shape}")
logging.warning(f"Shape before: {dfTemp.shape}")
#
dfTemp.sort_values(by=['Year', 'Season', 'City'], ascending=True, inplace=True, na_position='first')
dfTemp.drop_duplicates(keep='last', inplace=True)
dfTemp = dfTemp.reset_index(drop=True)
dfTemp = dfTemp.reset_index()
dfTemp.rename(columns={'index' : 'PK_Games'}, inplace=True)
dfTemp['PK_Games'] = dfTemp['PK_Games'] + 1
#
print(f"Shape after: {dfTemp.shape}")
logging.warning(f"Shape after: {dfTemp.shape}")
#
dfTemp['FK_Year'] = dfTemp['Year'].map(dfDimYear.set_index('Year')['PK_Year'])
dfTemp['FK_Season'] = dfTemp['Season'].map(dfDimSeason.set_index('Season')['PK_Season'])
dfTemp['FK_City'] = dfTemp['City'].map(dfDimCity.set_index('City')['PK_City'])
#
dfTemp = dfTemp.drop(columns=['Year', 'Season', 'City'])
#
outFile = 'Dim_Games.csv'
dfTemp.to_csv(outSchemaPath + outFile, index=False)
#
print(f"\nDimension Table created here:\n{outSchemaPath + outFile}")
logging.warning(f"\nDimension Table created here:\n{outSchemaPath + outFile}")
#
del dfTemp, dfDimSeason, dfDimCity, dfDimYear
#
#
#
## ################
## Create Dimension Table for Event       ################
##
#
## read in dimension tables for: Sport
sportDimFile = 'Dim_Sport.csv'
#
## load all columns into dataframe as string (object) by specifying the
##      dtype=str during loading, all the columns dtpye is object
#
dfDimSport = pd.read_csv(outSchemaPath+sportDimFile, sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
#
logging.warning(f"\n\n    *************************** \n    *************************** ")
logging.warning(f"\n    *************************** \n    *************************** \n\n")
#
print(f"\nACTION:\nCreating Dimension Table for column Sport.")
logging.warning(f"\nACTION:\nCreating Dimension Table for column Sport.")
#
dfTemp = dfin[['Event', 'Sport']]
#
print(f"Shape before: {dfTemp.shape}")
logging.warning(f"Shape before: {dfTemp.shape}")
#
dfTemp.sort_values(by=['Event', 'Sport'], ascending=True, inplace=True, na_position='first')
dfTemp.drop_duplicates(keep='last', inplace=True)
dfTemp = dfTemp.reset_index(drop=True)
dfTemp = dfTemp.reset_index()
dfTemp.rename(columns={'index' : 'PK_Event'}, inplace=True)
dfTemp['PK_Event'] = dfTemp['PK_Event'] + 1
#
print(f"Shape after: {dfTemp.shape}")
logging.warning(f"Shape after: {dfTemp.shape}")
#
dfTemp['FK_Sport'] = dfTemp['Sport'].map(dfDimSport.set_index('Sport')['PK_Sport'])
#
dfTemp = dfTemp.drop(columns=['Sport'])
#
outFile = 'Dim_Event.csv'
dfTemp.to_csv(outSchemaPath + outFile, index=False)
#
print(f"\nDimension Table created here:\n{outSchemaPath + outFile}")
logging.warning(f"\nDimension Table created here:\n{outSchemaPath + outFile}")
#
del dfTemp, dfDimSport
#
#
#
## ################
## Create Dimension Table for Athlete       ################
##
#
logging.warning(f"\n\n    *************************** \n    *************************** ")
logging.warning(f"\n    *************************** \n    *************************** \n\n")
#
print(f"\nACTION:\nCreating Dimension Table for column Athlete.")
logging.warning(f"\nACTION:\nCreating Dimension Table for column Athlete.")
#
dfTemp = dfin[['ID', 'Name', 'Sex', 'Height', 'Weight']]
#
print(f"Shape before: {dfTemp.shape}")
logging.warning(f"Shape before: {dfTemp.shape}")
#
dfTemp.sort_values(by=['Name'], ascending=True, inplace=True, na_position='first')
dfTemp.drop_duplicates(keep='last', inplace=True)
dfTemp = dfTemp.reset_index(drop=True)
dfTemp = dfTemp.reset_index()
dfTemp.rename(columns={'index' : 'PK_Athlete'}, inplace=True)
dfTemp['PK_Athlete'] = dfTemp['PK_Athlete'] + 1
#
print(f"Shape after: {dfTemp.shape}")
logging.warning(f"Shape after: {dfTemp.shape}")
#
outFile = 'Dim_Athlete.csv'
dfTemp.to_csv(outSchemaPath + outFile, index=False)
#
print(f"\nDimension Table created here:\n{outSchemaPath + outFile}")
logging.warning(f"\nDimension Table created here:\n{outSchemaPath + outFile}")
#
del dfTemp
#
print(f"\n\nDone")
#
#
#


ACTION:
Creating Dimension Table for column Season
Shape before: (255692, 1)
Shape after: (2, 2)
Dimension Table created here:
/home/rohit/PyWDUbuntu/DM2DataCuration/Schema/Dim_Season.csv

ACTION:
Creating Dimension Table for column City
Shape before: (255692, 1)


/home/rohit/anaconda3/envs/datacuration1/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/rohit/anaconda3/envs/datacuration1/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Shape after: (42, 2)
Dimension Table created here:
/home/rohit/PyWDUbuntu/DM2DataCuration/Schema/Dim_City.csv

ACTION:
Creating Dimension Table for column Year
Shape before: (255692, 1)
Shape after: (35, 2)
Dimension Table created here:
/home/rohit/PyWDUbuntu/DM2DataCuration/Schema/Dim_Year.csv

ACTION:
Creating Dimension Table for column Team
Shape before: (255692, 1)
Shape after: (1020, 2)
Dimension Table created here:
/home/rohit/PyWDUbuntu/DM2DataCuration/Schema/Dim_Team.csv

ACTION:
Creating Dimension Table for column Sport
Shape before: (255692, 1)
Shape after: (66, 2)
Dimension Table created here:
/home/rohit/PyWDUbuntu/DM2DataCuration/Schema/Dim_Sport.csv

ACTION:
Creating Dimension Table for column Event
Shape before: (255692, 1)
Shape after: (598, 2)
Dimension Table created here:
/home/rohit/PyWDUbuntu/DM2DataCuration/Schema/Dim_Event.csv

ACTION:
Creating Dimension Table for column Medal
Shape before: (255692, 1)
Shape after: (4, 2)
Dimension Table created here:
/home/rohit/

/home/rohit/anaconda3/envs/datacuration1/lib/python3.6/site-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/rohit/anaconda3/envs/datacuration1/lib/python3.6/site-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/rohit/anaconda3/envs/datacuration1/lib/python3.6/site-packages/ipykernel_launcher.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/rohit/anaconda3/en

Shape after: (134269, 6)

Dimension Table created here:
/home/rohit/PyWDUbuntu/DM2DataCuration/Schema/Dim_Athlete.csv


Done


In [33]:
## ################
## Create FACT Table for Participant       ################
##
#
## read in dimension tables for: Season, City, Year, Games, Sport, Team, Athlete, Event, NOC, Medals
seasonDimFile = 'Dim_Season.csv'
cityDimFile = 'Dim_City.csv'
yearDimFile = 'Dim_Year.csv'
gamesDimFile = 'Dim_Games.csv'
sportDimFile = 'Dim_Sport.csv'
teamDimFile = 'Dim_Team.csv'
athleteDimFile = 'Dim_Athlete.csv'
eventDimFile = 'Dim_Event.csv'
nocDimFile = 'Dim_NOC.csv'
medalDimFile = 'Dim_Medal.csv'
#
## load all columns into dataframe as string (object) by specifying the
##      dtype=str during loading, all the columns dtpye is object
#
dfDimSeason = pd.read_csv(outSchemaPath+seasonDimFile, sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
dfDimCity   = pd.read_csv(outSchemaPath+cityDimFile,   sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
dfDimYear   = pd.read_csv(outSchemaPath+yearDimFile,   sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
dfDimGames   = pd.read_csv(outSchemaPath+gamesDimFile,   sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
dfDimSport   = pd.read_csv(outSchemaPath+sportDimFile,   sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
dfDimTeam   = pd.read_csv(outSchemaPath+teamDimFile,   sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
dfDimAthlete   = pd.read_csv(outSchemaPath+athleteDimFile,   sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
dfDimEvent   = pd.read_csv(outSchemaPath+eventDimFile,   sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
dfDimNOC   = pd.read_csv(outSchemaPath+nocDimFile,   sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
dfDimMedal   = pd.read_csv(outSchemaPath+medalDimFile,   sep=',', dtype=str, header=0, \
                       keep_default_na=False, low_memory=False )
#
logging.warning(f"\n\n    *************************** \n    *************************** ")
logging.warning(f"\n    *************************** \n    *************************** \n\n")
#
print(f"\nACTION:\nCreating FACT Table for Participant.")
logging.warning(f"\nACTION:\nCreating FACT Table for Participant.")
#
dfTemp = dfin.copy()
#
print(f"Shape before: {dfTemp.shape}")
logging.warning(f"Shape before: {dfTemp.shape}")
#
dfTemp.sort_values(by=['Name'], ascending=True, inplace=True, na_position='first')
dfTemp.drop_duplicates(keep='last', inplace=True)
dfTemp = dfTemp.reset_index(drop=True)
dfTemp = dfTemp.reset_index()
dfTemp.rename(columns={'index' : 'PK_Participant'}, inplace=True)
dfTemp['PK_Participant'] = dfTemp['PK_Participant'] + 1
#
print(f"Shape after: {dfTemp.shape}")
logging.warning(f"Shape after: {dfTemp.shape}")
#
dfTemp['FK_Year'] = dfTemp['Year'].map(dfDimYear.set_index('Year')['PK_Year'])
dfTemp['FK_Season'] = dfTemp['Season'].map(dfDimSeason.set_index('Season')['PK_Season'])
dfTemp['FK_City'] = dfTemp['City'].map(dfDimCity.set_index('City')['PK_City'])
#
dfTemp['Fact_Concat_Year_Season_City'] = dfTemp['FK_Year'] + '_' + dfTemp['FK_Season'] + '_' + dfTemp['FK_City']
dfDimGames['GamesDim_Concat_Year_Season_City'] = dfDimGames['FK_Year']  + '_' + dfDimGames['FK_Season']  + '_' + dfDimGames['FK_City']
#
dfTemp['FK_Games'] = dfTemp['Fact_Concat_Year_Season_City'].map(dfDimGames.set_index('GamesDim_Concat_Year_Season_City')['PK_Games'])
dfTemp['FK_Event'] = dfTemp['Event'].map(dfDimEvent.set_index('Event')['PK_Event'])
dfTemp['FK_Team'] = dfTemp['Team'].map(dfDimTeam.set_index('Team')['PK_Team'])
dfTemp['FK_NOC'] = dfTemp['NOC'].map(dfDimNOC.set_index('NOC')['PK_NOC'])
dfTemp['FK_Athlete'] = dfTemp['ID'].map(dfDimAthlete.set_index('ID')['PK_Athlete'])
dfTemp['FK_Medal'] = dfTemp['Medal'].map(dfDimMedal.set_index('Medal')['PK_Medal'])
#
#dfTemp = dfTemp[['PK_Participant', 'FK_Games', 'FK_Event', 'FK_Team', 'FK_NOC', 'FK_Athlete', 'FK_Medal', 'Age']]
dfTemp = dfTemp[['PK_Participant', 'ID', 'Year', 'FK_Games', 'FK_Event', 'FK_Team', 'FK_NOC', 'FK_Athlete', 'FK_Medal', 'Age']]
dfTemp.rename(columns={'Age' : 'ParticipantAtheleteAge'}, inplace=True)
#
## has person won before -- ever or same sport
dfTemp['ParticipantWonBefore'] = '0'
dfWorkingWonBefore = dfTemp[['ParticipantWonBefore']].copy()
#
dfTemp1 = dfin[['ID', 'Year', 'Medal']]
dfTemp1 = dfTemp1.loc[dfTemp1['Medal'].isin(['gold', 'silver', 'bronze'])]
dfTemp1['YearNumeric'] = pd.to_numeric(dfTemp1['Year'], errors='coerce', downcast='integer')
dfTemp1.sort_values(by=['ID', 'YearNumeric'], ascending=True, inplace=True, na_position='first')
dfTemp['YearNumeric'] = pd.to_numeric(dfTemp['Year'], errors='coerce', downcast='integer')
#
i = -1
for row in dfTemp.itertuples():
    i += 1
    dictRow = row._asdict()
    dfEarlier = dfTemp1[dfTemp1['YearNumeric'] < dictRow['YearNumeric']]
    dfEarlier = dfEarlier[dfEarlier['ID'] == dictRow['ID']]
    if len(dfEarlier) > 0:
        dfWorkingWonBefore.loc[i] = '1'
dfTemp['ParticipantWonBefore'] = dfWorkingWonBefore['ParticipantWonBefore']
#
dfTemp = dfTemp.drop(columns=['ID', 'Year', 'YearNumeric'])
#
outFile = 'Fact_Participant.csv'
dfTemp.to_csv(outSchemaPath + outFile, index=False)
#
print(f"\nFact Table created here:\n{outSchemaPath + outFile}")
logging.warning(f"\nFact Table created here:\n{outSchemaPath + outFile}")
#
#del dfTemp, dfWorkingWonBefore, dfEarlier
del seasonDimFile, cityDimFile, yearDimFile, gamesDimFile, sportDimFile, teamDimFile
del athleteDimFile, eventDimFile, nocDimFile, medalDimFile
del dfDimSeason, dfDimCity, dfDimYear, dfDimGames, dfDimSport, dfDimTeam, dfDimAthlete
del dfDimEvent, dfDimNOC, dfDimMedal
#
print(f"\n\nDone")
#
#
#


ACTION:
Creating FACT Table for Participant.
Shape before: (255692, 16)
Shape after: (255692, 17)

Fact Table created here:
/home/rohit/PyWDUbuntu/DM2DataCuration/Schema/Fact_Participant.csv


Done
